![neurotic logo][logo]

# neurotic

_Curate, visualize, annotate, and share your behavioral ephys data using Python_

Check the [documentation][docs] for help with __neurotic__, including an [API reference guide][api] for the classes and functions used in this notebook.

[logo]: https://raw.githubusercontent.com/jpgill86/neurotic/master/neurotic/gui/icons/img/neurotic-logo-150.png
[docs]: https://neurotic.readthedocs.io/en/latest
[api]:  https://neurotic.readthedocs.io/en/latest/api.html

## Install Packages

In [ ]:
# check whether neurotic is currently installed
!pip show neurotic

In [ ]:
# install the latest release version
# !pip install -U neurotic

# alternatively, install the latest development version
# !pip install -U git+https://github.com/jpgill86/neurotic.git

## Import Packages

In [ ]:
import neurotic
import quantities as pq

## Select a Data Set

The next cell will download `metadata.yml`, a file containing information about available data sets and how they should be loaded, prepared, and displayed, including:
* locations of data files
* video synchronization parameters
* plot settings
* filter parameters
* amplitude windows for spike discrimination

In [ ]:
# download the metadata file
remote_metadata = 'https://raw.githubusercontent.com/jpgill86/neurotic/master/neurotic/example/metadata.yml'
neurotic.download(remote_metadata, './metadata.yml', overwrite_existing=False)

The `MetadataSelectorWidget` allows you to view the names and descriptions of data sets listed in `metadata.yml` and to select the one you would like to work with. Each data set may contain multiple files, such as an electrophysiology data file and a video file. Here is the key for icons found at the left of each entry:<br/>
&nbsp;&nbsp; ◆ &nbsp;&nbsp; all files can be found locally and none need to be downloaded<br/>
&nbsp;&nbsp; ⬖ &nbsp;&nbsp; some files can be found locally and others cannot<br/>
&nbsp;&nbsp; ◇ &nbsp;&nbsp; no files can be found locally and all need to be downloaded<br/>
&nbsp;&nbsp;&nbsp; ! &nbsp;&nbsp;&nbsp; the `video_offset` parameter is not set, which can cause out-of-sync video playback

__This example contains only one data set, so this part is easy!__

In [ ]:
# choose a dataset to load
metadata = neurotic.MetadataSelectorWidget(file='./metadata.yml')
display(metadata)

The next cell will download all of the data files associated with the selected data set. Unless `overwrite_existing=True`, a file will only be downloaded if a copy isn't found locally to avoid overwritting local changes.

In [ ]:
# download the data files
metadata.download_all_data_files(overwrite_existing=False)

## Load Data and Configure Ephyviewer

The next cell will use the settings contained in `metadata.yml` for the selected data set to do the following:
* read the electrophysiology data file
* apply filters to signals (`lazy=False` only)
* read annotations contained in CSV files
* run a simple spike detection algorithm using amplitude windows (`lazy=False` only)
* import spikes previously sorted by tridesclous
* calculate rectified area under the curve (RAUC) time series for each signal (`lazy=False` only)

When this is complete, a configuration widget will display that allows you to control which of the modular data viewers you would like the application to show. Click the "Launch" button to start up the application.

__Each time a new data set is selected using the `MetadataSelectorWidget`, you must rerun this cell.__

In [ ]:
# this cell must be rerun each time a new data set is selected above

# lazy loading takes advantage of Neo's high performance RawIO classes,
# which can read portions of files "on demand" to get just the data that
# is needed for plotting the currently visible time span
# - this decreases loading time and consumes much less memory, especially
#   for large files
# - to take advantage of these benefits, signal filtering, amplitude
#   window spike discrimination, and RAUC computation must be disabled
# - spike markers on signals are currently incompatible with lazy loading
lazy = False

blk = neurotic.load_dataset(metadata, lazy=lazy)

ephyviewer_config = neurotic.EphyviewerConfiguratorWidget(metadata, blk, lazy)
ephyviewer_config.show_all()
display(ephyviewer_config)

Executing the next cell is equivalent to pressing the "Launch" button and is provided for the convenience of running all cells in the notebook at once to start the application.

In [ ]:
# using the buttons above, select the viewers you'd like to see,
# then either click "Launch" or run this cell
ephyviewer_config.launch_ephyviewer()

After launching, you should see something like the following:

![example screenshot][screenshot]

[screenshot]: https://raw.githubusercontent.com/jpgill86/neurotic/master/docs/_static/example-screenshot.png